#### What is Ensemble Learning ?
##### Ensemble learning combines multiple models (often called "weak learners") to produce a stronger overall prediction model.
##### Analogy: It's like asking a group of friends (models) for advice instead of trusting just one—you get better decisions.
##### E.g. Read about Wisdom of crowds which means asking group of people who are from different domains / fields to get better answers.

##### There are four types in ensemble learning :
##### 1. Voting (Classification & Regression)
##### 2. Bagging i.e Bootstrap Aggregation (Using Random Forest Mostly Decision Trees) (Classification & Regression)
##### 3. Stacking (Classification & Regression)
##### 4. Boosting (Using ADABoost, XgBoost, Gradient Boosting) (Classification & Regression)

#### 1. Voting
##### Combine predictions from different models
##### Two types:
##### Hard Voting: Majority wins
##### Soft Voting: Average class probabilities
##### 🔧 When to Use:
##### You want a quick boost by mixing different model types
##### All models perform decently
##### ✅ Advantages:
##### Simple and intuitive
##### Leverages diverse model types
##### ❌ Disadvantages:
##### Doesn’t learn optimal weights
##### All models need to be trained separately


#### 2. Bagging (Bootstrap Aggregating)
##### Trains models on random subsets of data (with replacement)
##### Combines results by averaging (regression) or voting (classification)
##### ✅ Algorithm: Random Forest
##### 🔧 When to Use:
##### You have a model that overfits (high variance)
##### You want fast, robust performance
##### ✅ Advantages:
##### Reduces overfitting
##### Simple to parallelize
##### Robust to noise and outliers
##### ❌ Disadvantages:
##### Less interpretability
##### Doesn’t reduce bias


#### 3. Stacking
##### Train multiple base models
##### Their predictions become input to a meta-model and also weights are provided for better results.
##### It's like feeding model outputs into another model that learns to blend them better.
##### 🔧 When to Use:
##### You want to blend strengths of multiple models
##### You've tried Voting but want more learning power
##### ✅ Advantages:
##### Learns optimal model combination
##### Usually performs better than voting
##### ❌ Disadvantages:
##### Harder to implement
##### Takes more time to train


#### 4. Boosting
##### Builds models sequentially
##### Each new model learns from the errors of the previous
##### Focuses more on harder cases
##### 🔧 Algorithms:
##### AdaBoost (adaptive weighting)
##### Gradient Boosting (residual learning)
##### XGBoost, LightGBM, CatBoost (optimized versions)
##### ✅ When to Use:
##### You need very high accuracy
##### You’re competing in Kaggle or working with complex patterns
##### ✅ Advantages:
##### Reduces bias and variance
##### Works well with less data
##### Often achieves state-of-the-art results
##### ❌ Disadvantages:
##### Sensitive to noisy data
##### Slower training time
##### Harder to tune

### Voting

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, RandomForestRegressor, VotingRegressor, BaggingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, SVR
from sklearn.datasets import load_breast_cancer,fetch_california_housing,load_digits
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_absolute_error, mean_squared_error, r2_score

In [2]:
#using breast cancer dataset to predict tumpor (classification problem)
breast_cancer_data = load_breast_cancer()
breast_cancer_data

{'data': array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
         1.240e-01],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0

In [3]:
#getting target and training values
X = pd.DataFrame(breast_cancer_data.data, columns=breast_cancer_data.feature_names)
#converting it to 1d array for that using Series
y = pd.Series(breast_cancer_data.target, name='target')

In [4]:
X

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [5]:
y

0      0
1      0
2      0
3      0
4      0
      ..
564    0
565    0
566    0
567    0
568    1
Name: target, Length: 569, dtype: int32

In [6]:
#splitting the data
X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

#Scaler to scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#use classification models
lr = LogisticRegression()
rf = RandomForestClassifier()
svc = SVC(probability=True)

#creating estimators
estimators = [('lr', lr),('rf', rf),('svc', svc)]

#Using hard and soft voting (by default hard is used if not specified) 
hard_voting = VotingClassifier(estimators=estimators, voting='hard')
soft_voting = VotingClassifier(estimators=estimators, voting='soft')
hard_voting.fit(X_train,y_train)
soft_voting.fit(X_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()),
                             ('svc', SVC(probability=True))],
                 voting='soft')

In [7]:
#predicting voting results
y_pred_hard = hard_voting.predict(X_test)
y_pred_soft = soft_voting.predict(X_test)

print(f"Accuracy Score for Hard Voting : {accuracy_score(y_test, y_pred_hard)*100:.2f}")
print(f"Accuracy Score for Soft Voting : {accuracy_score(y_test, y_pred_soft)*100:.2f}")
print(f"Confusion Matrix for Soft Voting : {confusion_matrix(y_test, y_pred_soft)}")
print(f"Confusion Matrix for Hard Voting : {confusion_matrix(y_test, y_pred_hard)}")
print(f"Classification Report for Soft Voting : {classification_report(y_test, y_pred_soft)}")
print(f"Classification Report for Hard Voting : {classification_report(y_test, y_pred_hard)}")

Accuracy Score for Hard Voting : 98.25
Accuracy Score for Soft Voting : 97.37
Confusion Matrix for Soft Voting : [[41  2]
 [ 1 70]]
Confusion Matrix for Hard Voting : [[41  2]
 [ 0 71]]
Classification Report for Soft Voting :               precision    recall  f1-score   support

           0       0.98      0.95      0.96        43
           1       0.97      0.99      0.98        71

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114

Classification Report for Hard Voting :               precision    recall  f1-score   support

           0       1.00      0.95      0.98        43
           1       0.97      1.00      0.99        71

    accuracy                           0.98       114
   macro avg       0.99      0.98      0.98       114
weighted avg       0.98      0.98      0.98       114



In [8]:
#Using california housing predicition dataset to predict prices (Regression Problem)
data = fetch_california_housing()
data

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [9]:
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name='target')

In [10]:
X

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25
...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32


In [11]:
y

0        4.526
1        3.585
2        3.521
3        3.413
4        3.422
         ...  
20635    0.781
20636    0.771
20637    0.923
20638    0.847
20639    0.894
Name: target, Length: 20640, dtype: float64

In [12]:
#split data
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

#scale data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#use regression models
lr = LinearRegression()
rfr = RandomForestRegressor()
svr = SVR()

#create estimators
estimators = [('lr', lr), ('rfr', rfr), ('svr',svr)]

#using voting regressor
voting_reg = VotingRegressor(estimators=estimators)
voting_reg.fit(X_train,y_train)

VotingRegressor(estimators=[('lr', LinearRegression()),
                            ('rfr', RandomForestRegressor()), ('svr', SVR())])

In [13]:
#predicting results
y_pred = voting_reg.predict(X_test)

#predicting mean_absolute_error, r2 score, mean_squared_error
print(f"MAE Score: {mean_absolute_error(y_test, y_pred)*100:.2f}")
print(f"MSE Score: {mean_squared_error(y_test, y_pred)*100:.2f}")
print(f"R2 Score: {r2_score(y_test, y_pred)*100:.2f}")

MAE Score: 38.77
MSE Score: 32.75
R2 Score: 75.01


### Bagging

In [14]:
#using digits dataset to perform classification
digits_data = load_digits()
digits_data

{'data': array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ..., 10.,  0.,  0.],
        [ 0.,  0.,  0., ..., 16.,  9.,  0.],
        ...,
        [ 0.,  0.,  1., ...,  6.,  0.,  0.],
        [ 0.,  0.,  2., ..., 12.,  0.,  0.],
        [ 0.,  0., 10., ..., 12.,  1.,  0.]]),
 'target': array([0, 1, 2, ..., 8, 9, 8]),
 'frame': None,
 'feature_names': ['pixel_0_0',
  'pixel_0_1',
  'pixel_0_2',
  'pixel_0_3',
  'pixel_0_4',
  'pixel_0_5',
  'pixel_0_6',
  'pixel_0_7',
  'pixel_1_0',
  'pixel_1_1',
  'pixel_1_2',
  'pixel_1_3',
  'pixel_1_4',
  'pixel_1_5',
  'pixel_1_6',
  'pixel_1_7',
  'pixel_2_0',
  'pixel_2_1',
  'pixel_2_2',
  'pixel_2_3',
  'pixel_2_4',
  'pixel_2_5',
  'pixel_2_6',
  'pixel_2_7',
  'pixel_3_0',
  'pixel_3_1',
  'pixel_3_2',
  'pixel_3_3',
  'pixel_3_4',
  'pixel_3_5',
  'pixel_3_6',
  'pixel_3_7',
  'pixel_4_0',
  'pixel_4_1',
  'pixel_4_2',
  'pixel_4_3',
  'pixel_4_4',
  'pixel_4_5',
  'pixel_4_6',
  'pixel_4_7',
  'pixel_5_0',
  'pixel_5_1',
 

In [15]:
#split training and target values
X = pd.DataFrame(digits_data.data, columns=digits_data.feature_names)
y = pd.Series(digits_data.target, name='target')

In [16]:
X

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_6,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,pixel_7_7
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,9.0,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1792,0.0,0.0,4.0,10.0,13.0,6.0,0.0,0.0,0.0,1.0,...,4.0,0.0,0.0,0.0,2.0,14.0,15.0,9.0,0.0,0.0
1793,0.0,0.0,6.0,16.0,13.0,11.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,6.0,16.0,14.0,6.0,0.0,0.0
1794,0.0,0.0,1.0,11.0,15.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,9.0,13.0,6.0,0.0,0.0
1795,0.0,0.0,2.0,10.0,7.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,5.0,12.0,16.0,12.0,0.0,0.0


In [17]:
y

0       0
1       1
2       2
3       3
4       4
       ..
1792    9
1793    0
1794    8
1795    9
1796    8
Name: target, Length: 1797, dtype: int32

In [19]:
#splitting data
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

#scaling data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#Using Classification Models 
# The whole idea of bagging (Bootstrap Aggregating) is to train multiple instances of the same model on different subsets of the data (with replacement).
# lr = LogisticRegression()
rf = RandomForestClassifier()
# svc = SVC(probability=True)

#use bagging
bag_class = BaggingClassifier(estimator=rf, bootstrap=True, max_samples=0.25, n_jobs=-1)
bag_class.fit(X_train,y_train)

#predict results
y_pred = bag_class.predict(X_test)

print(f"Accuracy Score: {accuracy_score(y_test, y_pred)*100:.2f}")
print(f"Confusion Matrix: {confusion_matrix(y_test, y_pred)}")
print(f"Classification Report: {classification_report(y_test, y_pred)}")

Accuracy Score: 95.83
Confusion Matrix: [[33  0  0  0  0  0  0  0  0  0]
 [ 0 27  0  0  0  0  0  0  0  1]
 [ 0  0 33  0  0  0  0  0  0  0]
 [ 0  0  0 33  0  0  0  0  1  0]
 [ 0  0  0  0 46  0  0  0  0  0]
 [ 0  0  0  0  0 42  1  0  0  4]
 [ 1  0  0  0  0  0 34  0  0  0]
 [ 0  0  0  0  0  0  0 33  0  1]
 [ 0  2  0  0  0  1  0  0 27  0]
 [ 0  0  0  0  0  1  0  2  0 37]]
Classification Report:               precision    recall  f1-score   support

           0       0.97      1.00      0.99        33
           1       0.93      0.96      0.95        28
           2       1.00      1.00      1.00        33
           3       1.00      0.97      0.99        34
           4       1.00      1.00      1.00        46
           5       0.95      0.89      0.92        47
           6       0.97      0.97      0.97        35
           7       0.94      0.97      0.96        34
           8       0.96      0.90      0.93        30
           9       0.86      0.93      0.89        40

    accura

In [20]:
#using concrete dataset for regression
df = pd.read_csv('Concrete.csv')
df.sample(5)

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
284,181.4,0.0,167.0,169.6,7.6,1055.6,777.8,3,13.62
191,233.8,0.0,94.6,197.9,4.6,947.0,852.2,28,22.84
781,296.0,0.0,0.0,192.0,0.0,1085.0,765.0,7,14.20
681,173.0,116.0,0.0,192.0,0.0,946.8,856.8,28,24.28
190,233.8,0.0,94.6,197.9,4.6,947.0,852.2,14,22.14


In [40]:
#split target
X = df.iloc[:, 0:8]
y = df.iloc[:, -1]

# #train split the data
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# #scale the data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# #create Bagging Regressor model
bag_reg = BaggingRegressor(estimator=RandomForestRegressor(),max_samples=0.25,n_jobs=-1,bootstrap=True)
bag_reg.fit(X_train,y_train)

BaggingRegressor(estimator=RandomForestRegressor(), max_samples=0.25, n_jobs=-1)

In [41]:
#predicting results
y_pred = bag_reg.predict(X_test)

#predicting mean_absolute_error, r2 score, mean_squared_error
print(f"MAE Score: {mean_absolute_error(y_test, y_pred)*100:.2f}")
print(f"MSE Score: {mean_squared_error(y_test, y_pred)*100:.2f}")
print(f"R2 Score: {r2_score(y_test, y_pred)*100:.2f}")

MAE Score: 533.52
MSE Score: 4603.04
R2 Score: 82.14
